In [2]:
!pip install tensorflow

In [22]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv(r"/content/drive/MyDrive/cmi-detect-behavior-with-sensor-data/train.csv")

In [7]:
sequences = df.groupby("sequence_id")

In [10]:
y = df.groupby("sequence_id")["gesture"].first().values
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y)

In [11]:
needCols = [i for i in df.columns if i.startswith("acc_") or i.startswith("rot_") or i.startswith("thm_") or i.startswith("tof_")]

In [12]:
df[needCols].isna().sum()

,0
acc_x,0
acc_y,0
acc_z,0
rot_w,3692
rot_x,3692
...,...
tof_5_v59,30142
tof_5_v60,30142
tof_5_v61,30142
tof_5_v62,30142


In [13]:
X = []
Xlen = []

In [14]:
def func(data, colss):
  data = data[colss].copy()
  data = data.fillna(-2)
  scaler = StandardScaler()
  data = scaler.fit_transform(data)
  return data

In [15]:
for i, j in sequences:
  processed = func(j, needCols)
  X.append(processed)
  Xlen.append(processed.shape[0])

In [16]:
pad_len = int(np.percentile(Xlen, 90))
pad_len

103

In [17]:
y = np.array(y)
y.shape

(8151, 18)

In [18]:
X = pad_sequences(X, maxlen=pad_len, dtype="float64", padding="post", truncating="post")

In [19]:
X.shape, y.shape

((8151, 103, 332), (8151, 18))

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)